![Alt text](images/banner.png)

## Deploy the Saved Model in the project to Deployment Space

### Initial Setup

Some initial setup specific to running this notebook as part of the pipeline.

In [ ]:
import os
#This environment variable is automatically set in WS Pipelines and are needed to access various services.
TOKEN = os.getenv("USER_ACCESS_TOKEN")

In [ ]:
if os.getenv("running_in_production_pipeline"):
    running_in_production_pipeline = True
    # If you want to run additional steps when deploying to production like reporting to external services, you can use this variable to trigger that
    # It can also be used to skip steps that are only needed in development like plotting
    print("notebook is running in a production pipeline!")
else:
    running_in_production_pipeline = False
    print("notebook is running in a development enviroment!")

In [26]:
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
from ibm_watson_machine_learning import APIClient
# from ibm_aigov_facts_client import AIGovFactsClient #removing due to current issues --> put back ASAP
from ibm_watson_studio_pipelines import WSPipelines
from botocore.client import Config
import ibm_boto3
import pandas as pd
import json
import os
import requests
import pickle
import vars_and_utils as vars_and_utils


## Instantiate WML Client

In [ ]:
WML_CREDENTIALS = {
   "token": TOKEN,
   "instance_id" : "openshift",
   "url": os.environ['RUNTIME_ENV_APSX_URL'],
   "version": "4.6"
}

wml_client = APIClient(WML_CREDENTIALS)

In [ ]:
if running_in_production_pipeline:
    deployment_space_id=vars_and_utils.deployment_space_id_PROD
else:
    deployment_space_id=vars_and_utils.deployment_space_id_DEV
    
deployment_space_id
    

In [ ]:
wml_client.set.default_space(deployment_space_id)

# Deserialize model

TODO: Later get model from model inventory instead

In [ ]:
with open(vars_and_utils.model_path, 'rb') as f:
    model = pickle.load(f)

### Load Sample Data 

In [ ]:
payload_data = vars_and_utils.load_data_from_filesystem(vars_and_utils.test_data_path)
payload_data = payload_data.drop('Risk',axis=1)
fields = payload_data.columns.tolist()
values = [payload_data.values.tolist()[0]]

payload_scoring = {"input_data": [{"fields": fields, "values": values}]}
json.dumps(payload_scoring)

In [ ]:
# TODO: Move to notebook 3 and skip (de)serialization process
software_spec_uid = wml_client.software_specifications.get_id_by_name("runtime-22.2-py3.10")
model_props_gbt = {
    wml_client.repository.ModelMetaNames.NAME: vars_and_utils.model_name,
    wml_client.repository.ModelMetaNames.DESCRIPTION: vars_and_utils.model_name,
    wml_client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: software_spec_uid,
    wml_client.repository.ModelMetaNames.TYPE: "scikit-learn_1.1"
}

published_model_details = wml_client.repository.store_model(model=model, meta_props=model_props_gbt, training_data=fields,training_target=values)
print(published_model_details)

In [ ]:
model_id = wml_client.repository.get_model_id(published_model_details)
model_id

In [ ]:
# There is an issue with WML Deployments right now. TODO: fix this
# This is the ID of the model that was uploaded manually --> that works
model_id = "65c060dc-ea5b-4c48-86fe-acb97853b5df"
# This is fake and we need to fix this!

## Promote the Model to deployment space and Deploy the Model

In [ ]:
meta_data = {
    wml_client.deployments.ConfigurationMetaNames.NAME: vars_and_utils.deployment_name,
    wml_client.deployments.ConfigurationMetaNames.ONLINE: {},
    wml_client.deployments.ConfigurationMetaNames.HARDWARE_SPEC: {
        "name": "S",
        "num_nodes": 1,
    }
}

deployment_details = wml_client.deployments.create(model_id, meta_props=meta_data)

In [ ]:
deployment_uid = wml_client.deployments.get_id(deployment_details)
deployment_uid

## Score the Endpoint

### Model Testing on the Serving Endpoint



In [ ]:
predictions = wml_client.deployments.score(deployment_uid, payload_scoring)
predictions

### Test for Downstream Apps without using WML SDK.

In [ ]:
# deploy_done is true if deployment_uid and model_id are not null
deploy_done = bool(deployment_uid) and bool(model_id)

## Register the output variables for the next pipeine stage
every notebook outputs a "was_successful" boolean variable. The logic behind this is different for every notebook and can be altered to fit the needs of the project.
If needed additional variables can be created here but they also need to registered as output variables in the Watson Pipelines UI.

In [ ]:
deployment_done = {}
deployment_done['was_succesfull'] = deploy_done
deployment_done['deployment_id'] = deployment_uid
deployment_done['model_id'] = model_id

pipelines_client = WSPipelines.from_token(TOKEN)
pipelines_client.store_results(deployment_done)